# Sistema Multiagentes para Criação de Roteiros de Viagem 🏖️

Criaremos um conjunto de agentes (crew) para definir um roteiro de viagem para um lugar escolhido, que tenha um bom custo-benefício de estadia e alimentação sendo também romantico para casais.

### **Sobre o CrewAI**:
> CrewAI é um framework de orquestração de IA de código aberto que permite a criação de *"tripulações" (crews)* de agentes autônomos que colaboram para completar tarefas complexas.

> Ele usa uma estrutura modular onde cada agente tem uma missão, ferramentas e uma função específica, e as tarefas podem ser delegadas entre eles. O objetivo é automatizar fluxos de trabalho e otimizar a resolução de problemas, simulando uma equipe de trabalho humana com agentes de IA.



# 1. Instalação do CREWAI


* O ```!``` é só pra dizer que é um comando de terminal.

* ```pip install crewai``` baixa e instala a lib.

* O ```--quiet``` deixa tudo mais silencioso, sem aquele monte de texto na tela.




In [1]:
!pip install --quiet crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 641.8/641.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 10.7 MB/s e

# 2. Importação das dependências e variáveis de ambiente


* Estamos importando as peças principais do crewai.

  `from crewai import Agent, Task, Crew`

  * Agent → o “agente”, quem faz as tarefas.
  * Task → a tarefa que ele precisa executar.
  * Crew → o grupo que junta tudo e coordena a brincadeira.


* Importando o módulo do Colab para pegar as variáveis de ambiente.
  
  `from google.colab import userdata`

* Pegamos a chave da API do Gemini que foi salva no Colab.
  
  `GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')`


In [3]:
from crewai import Agent, Task, Crew
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

## Importamos também o GEMINI, modelo LLM da Google.

> Por padrão o CREWAI utiliza o GPT da OpenAI. Mas o Gemini tem mais créditos gratuitos pra nós meras mortais.

In [4]:
from crewai import LLM

gemini_llm = LLM(
    model="gemini/gemini-2.5-flash",
    api_key=GEMINI_API_KEY,
    temperature=0.0
)

# 2. Definição do Agente

Armazenamos em uma variável a entidade `Agent` para criar nosso primeiro agente, suas características e atribuições.

Aqui vamos usar os seguintes parâmetros:

1. **`role`**

É o *papel* do agente, o “título do cargo”.

2. **`goal`**

É o **objetivo principal** do agente — aquilo que ele sempre tenta alcançar.
Utilizamos variáveis (`{destino}` e `{tempo}`) para preencher na hora de executar o agente.

3. **`backstory`**

  É o **contexto**, a história por trás do agente. "Passado do vilão".
Vai guiar o modelo a ter um tom específico ou considerar responder como alguem com esse contexto responderia.

4. **`llm`**

  Aqui dizemos pra ele qual LLM usar, por default é o GPT mas vamos usar o GEMINI.

In [5]:
guia_turistico = Agent(
    role="Guia Turistico",
    goal="Entregar um roteiro de viagem para {destino} com o melhor custo benefício dado tempo {tempo} de férias.",
    backstory="Você é um guia turistico com 10 anos de experiência especialista em viagens para casais.",
    llm=gemini_llm
)

# 3. Definição da Tarefa

Agora armazenamos em uma variável a entidade `Task` para criar estrutura e regras que nosso agente vai executar na hora de tocar essa Tarefa.

Os parâmetros que usamos são:

1. **`agent`**

Quem vai executar essa tarefa.  

2. **`description`**

É onde você explica passo a passo o que você quer que o agente faça.

3. **`expected_output`**

Aqui você define como quer que a resposta final venha formatada.

In [26]:
escrever_roteiro = Task(
    agent=guia_turistico,
    description="""
    Escreva um roteiro de viagem para {destino}.
    Considere as melhores opções de restaurante considerando o melhor custo benefício.
    Inclua também opções de experiências nativas do local {destino}.
    E encaixe tudo no tempo disponível que é {tempo}.
    """,
    expected_output="Lista de restaurantes e experiências com horários definidos para cada um."
)

# 4. Definição do CREW


1. **`agents`**

Aqui você lista[] quais agentes fazem parte da equipe. Montamos o time.

2. **`tasks`**

Uma lista[] de quais tarefas esse time precisa executar.

3. **`verbose=True`**

Esse parâmetro serve pra mostrar pra gente o que está acontecendo por trás dos panos.

In [8]:
crew = Crew(
    agents=[guia_turistico],
    tasks=[escrever_roteiro],
    verbose=True
)

# 5. Fazendo funcionar pela primeira vez

Vamos rodar nosso crew pela primeira vez.
Esperamos que ao fim do pipeline, ele seja capaz de nos entregar uma lista com um belo roteiro de viagem seguindo o que pedimos para ele executar como Task.

Você lembra que na definição da tarefa e do agente colocamos entre chaves `{ }` os parâmetros que passariamos porteriormente para o Agente? Essa é a hora.

In [9]:
payload = {
    "destino": "Alagoas",
    "tempo": "01 semana"
}

roteiro_pronto = crew.kickoff(inputs=payload)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 39cb846b-fe58-407b-a9be-948bde4ea028                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Guia Turistico                                                                                          │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Escreva um roteiro de viagem para Alagoas.                                                                 │
│      Considere as melhores opções de restaurante considerando o melhor custo benefício.                         │
│      Inclua também opções de experiências nativas do local Alagoas.                                             │
│      E encaixe tudo no tempo disponível que é 01 semana.                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Guia Turistico                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Olá, casal apaixonado! Que alegria poder guiá-los por Alagoas, um paraíso que conheço como a palma da minha    │
│  mão há 10 anos, especialista em criar memórias inesquecíveis para casais. Preparem-se para uma semana de       │
│  encantos, sabores e experiências autênticas, tudo pensado para oferecer o melhor custo-benefício e otimizar    │
│  cada minuto das suas férias.                                                                                   │
│                                                                                                                 │
│  Este roteiro foi cuidadosamente elaborado para que vocês vivam Alagoas intensamente, com momentos de           │
│  aventura, relaxamento e muita gastronomia local, sem pesar no bolso.                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## Roteiro de Viagem para Casais em Alagoas: 01 Semana de Encantos e Sabores                                   │
│                                                                                                                 │
│  **Base de Hospedagem Sugerida:** Maceió (Pajuçara, Ponta Verde ou Jatiúca) – áreas com excelente               │
│  infraestrutura, segurança e fácil acesso aos passeios.                                                         │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **Dia 1: Chegada em Maceió e Aclimatação à Orla Mais Bonita do Brasil**                                    │
│                                                                                                                 │
│  *   **Manhã (09:00 - 12:00):** Chegada ao Aeroporto Internacional Zumbi dos Palmares (MCZ). Traslado           │
│  privativo ou por aplicativo para o hotel em Maceió. Check-in e acomodação.                                     │
│  *   **Almoço (13:00 - 14:30):** **Restaurante Bodega do Sertão** (Av. Dr. Júlio Marques Luz, 220 - Jatiúca).   │
│      *   **Custo-benefício:** Excelente. Um buffet nordestino farto e delicioso, com ambiente temático que já   │
│  é uma atração à parte. Perfeito para mergulhar na culinária local.                                             │
│      *   **Experiência Nativa:** Culinária regional autêntica, com pratos como carne de sol, baião de dois,     │
│  macaxeira e doces caseiros.                                                                                    │
│  *   **Tarde (15:30 - 18:00):** Caminhada relaxante pela orla de **Ponta Verde e Pajuçara**. Aproveitem para    │
│  sentir a brisa, admirar o mar e tirar as primeiras fotos.                                                      │
│  *   **Pôr do Sol (17:30 - 18:00):** Escolham um dos charmosos quiosques na orla para apreciar o espetáculo do  │
│  pôr do sol alagoano.                                  

# 6. Vendo o resultado do Agente de Viagens.

No print abaixo pudemos ver que o Agente ficou inspirado com a nossa viagem e nos deu todas as dicas para um roteiro relevante de viagens.

Isso apenas baseado no que ele tem disponível dentro da LLM para trabalhar.

In [10]:
print(roteiro_pronto)

Olá, casal apaixonado! Que alegria poder guiá-los por Alagoas, um paraíso que conheço como a palma da minha mão há 10 anos, especialista em criar memórias inesquecíveis para casais. Preparem-se para uma semana de encantos, sabores e experiências autênticas, tudo pensado para oferecer o melhor custo-benefício e otimizar cada minuto das suas férias.

Este roteiro foi cuidadosamente elaborado para que vocês vivam Alagoas intensamente, com momentos de aventura, relaxamento e muita gastronomia local, sem pesar no bolso.

---

## Roteiro de Viagem para Casais em Alagoas: 01 Semana de Encantos e Sabores

**Base de Hospedagem Sugerida:** Maceió (Pajuçara, Ponta Verde ou Jatiúca) – áreas com excelente infraestrutura, segurança e fácil acesso aos passeios.

---

### **Dia 1: Chegada em Maceió e Aclimatação à Orla Mais Bonita do Brasil**

*   **Manhã (09:00 - 12:00):** Chegada ao Aeroporto Internacional Zumbi dos Palmares (MCZ). Traslado privativo ou por aplicativo para o hotel em Maceió. Check-i

Mas, e se ele tivesse acesso a internet para sugerir coisas ainda mais relevantes que pedirmos?

# 7. Turbinando nosso Crew com o uso de TOOLS (ferramentas)

Vamos usar o SerperDev para fazer com que o nosso Agente pesquise na internet ao invés de contar com a resposta treinada do conhecimento da LLM.

In [19]:
!pip install --quiet crewai[tools]

In [38]:
from crewai_tools import SerperDevTool
import os

search_tool = SerperDevTool()
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [39]:
guia_turistico_turbinado = Agent(
    role="Guia Turistico",
    goal="""
    Entregar um roteiro de viagem para {destino} com as melhores indicações e lugares atualizados.
    Pesquise na internet restaurantes e pousadas com melhor custo benefício em {destino}
    dado tempo {tempo} de férias.
    """,
    backstory="Você é um guia turistico com 10 anos de experiência especialista em viagens para casais.",
    llm=gemini_llm,
    tools=[search_tool]
)

Perceba que agora temos uma `tool` dentro dos parâmetros dos nossos agentes.

E também inclumos a direção dentro do goal e da description da Task para que ele se utilize da internet.

In [40]:
escrever_roteiro_turbinado = Task(
    agent=guia_turistico_turbinado,
    description="""
    Escreva um roteiro de viagem para {destino}.
    Considere as melhores opções de restaurante pesquise na internet o melhor custo benefício.
    Considere apenas locais com reviews acima de 3,5 estrelas.
    Pesquisa também opções de experiências nativas do local {destino}.
    E encaixe tudo no tempo disponível que é {tempo}.
    """,
    expected_output="Lista de restaurantes e experiências com horários definidos para cada um.",
    tools=[search_tool]
)

In [41]:
crew_turbinado = Crew(
    agents=[guia_turistico_turbinado],
    tasks=[escrever_roteiro_turbinado],
    verbose=True
)

In [42]:
payload = {
    "destino": "Alagoas",
    "tempo": "01 semana"
}

roteiro_turbinado_pronto = crew_turbinado.kickoff(inputs=payload)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0639939b-b49f-47ee-9b32-47a4342ab2ad                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Guia Turistico                                                                                          │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Escreva um roteiro de viagem para Alagoas.                                                                 │
│      Considere as melhores opções de restaurante pesquise na internet o melhor custo benefício.                 │
│      Considere apenas locais com reviews acima de 3,5 estrelas.                                                 │
│      Pesquisa também opções de experiências nativas do local Alagoas.                                           │
│      E encaixe tudo no tempo disponível que é 01 semana.                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Guia Turistico                                                                                          │
│                                                                                                                 │
│  Thought: Action: Search the internet with Serper                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Guia Turistico                                                                                          │
│                                                                                                                 │
│  Thought: Thought: Para Maragogi, Pousada Verdes Mares, Pousada Encontro das Águas e Pousada Olho D'água        │
│  parecem ter boas avaliações e custo-benefício.                                                                 │
│  Agora, vou pesquisar pousadas em São Miguel dos Milagres e, em seguida, experiências nativas em Alagoas.       │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Guia Turistico                                                                                          │
│                                                                                                                 │
│  Thought: Thought: Para São Miguel dos Milagres, Angá Beach Hotel, Wassu Pousada, Pousada Paraíso dos Milagres  │
│  e Pousada Corália são opções com boas avaliações e menções de custo-benefício.                                 │
│  Agora, preciso focar nas experiências nativas de Alagoas.                                                      │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [43]:
print(roteiro_turbinado_pronto)

**Roteiro de Viagem para Casais em Alagoas (01 Semana)**

Olá, casal! Com meus 10 anos de experiência em roteiros românticos, preparei um itinerário especial para vocês explorarem as belezas de Alagoas em uma semana, com foco em custo-benefício, experiências autênticas e, claro, muito romance. Todas as sugestões de restaurantes e pousadas possuem avaliações acima de 3.5 estrelas.

---

**Hospedagem Sugerida (Custo-Benefício e Avaliações > 3.5):**

*   **Maceió:**
    *   **Pousada Praia Pajuçara:** Localização excelente, a poucos passos da praia e da feirinha de artesanato. Quartos confortáveis e bom café da manhã. (Avaliação média 8.7 no Booking.com)
    *   **Kyros Pousada:** Próxima à praia de Ponta Verde, oferece um bom custo-benefício com quartos limpos e atendimento atencioso. (Avaliação média 8.5 no Booking.com)
*   **Maragogi:**
    *   **Pousada Verdes Mares:** Bem localizada, com piscina e quartos aconchegantes, ideal para casais. (Avaliação 4.5 no Tripadvisor)
    *   **Pous

In [44]:
print(roteiro_pronto)

Olá, casal apaixonado! Que alegria poder guiá-los por Alagoas, um paraíso que conheço como a palma da minha mão há 10 anos, especialista em criar memórias inesquecíveis para casais. Preparem-se para uma semana de encantos, sabores e experiências autênticas, tudo pensado para oferecer o melhor custo-benefício e otimizar cada minuto das suas férias.

Este roteiro foi cuidadosamente elaborado para que vocês vivam Alagoas intensamente, com momentos de aventura, relaxamento e muita gastronomia local, sem pesar no bolso.

---

## Roteiro de Viagem para Casais em Alagoas: 01 Semana de Encantos e Sabores

**Base de Hospedagem Sugerida:** Maceió (Pajuçara, Ponta Verde ou Jatiúca) – áreas com excelente infraestrutura, segurança e fácil acesso aos passeios.

---

### **Dia 1: Chegada em Maceió e Aclimatação à Orla Mais Bonita do Brasil**

*   **Manhã (09:00 - 12:00):** Chegada ao Aeroporto Internacional Zumbi dos Palmares (MCZ). Traslado privativo ou por aplicativo para o hotel em Maceió. Check-i